In [1]:
import pandas as pd
import re

LOAD CORP LIST

In [2]:
corp_df = pd.read_csv('../../data/preprocessed/상장법인목록.csv',
                      dtype=object)
corp_df.shape

(2542, 9)

In [3]:
corp_df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,탈로스,434190,일차전지 및 축전지 제조업,군용 리튬이온 이차전지,2022-10-24,12월,채재호,http://www.talos.or.kr,경기도
1,플라즈맵,405000,의료용 기기 제조업,플라즈마 멸균기 및 표면처리기기,2022-10-21,12월,임유봉,http://plasmapp.com/,대전광역시
2,엔에이치스팩24호,437780,금융 지원 서비스업,기업인수합병,2022-10-20,12월,이인직,NaN,서울특별시
3,핀텔,291810,소프트웨어 개발 및 공급업,AI 영상 분석 솔루션 및 소프트웨어,2022-10-20,12월,김동기,NaN,경기도
4,하나금융25호스팩,435620,금융 지원 서비스업,기업인수합병,2022-10-20,12월,남강욱,NaN,서울특별시


In [4]:
corp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   회사명     2542 non-null   object
 1   종목코드    2542 non-null   object
 2   업종      2542 non-null   object
 3   주요제품    2527 non-null   object
 4   상장일     2542 non-null   object
 5   결산월     2542 non-null   object
 6   대표자명    2542 non-null   object
 7   홈페이지    2377 non-null   object
 8   지역      2541 non-null   object
dtypes: object(9)
memory usage: 178.9+ KB


In [5]:
corp_df.describe()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
count,2542,2542,2542,2527,2542,2542,2542,2377,2541
unique,2542,2542,161,2358,1886,9,2418,2355,22
top,탈로스,434190,소프트웨어 개발 및 공급업,기업인수합병,1976-06-30,12월,-,http://www.ibks.com,서울특별시
freq,1,1,158,32,8,2482,5,3,1036


LOAD CLASS

In [6]:
class_df = pd.read_excel('../../data/origin/kostat/한국표준산업분류(10차)_표.xlsx',
                         usecols=range(1, 6, 2), skiprows=2)
class_df = class_df.drop_duplicates()
class_df = class_df.fillna(method='ffill')
class_df = class_df.drop_duplicates()
class_df = class_df.replace(r'\(.+\)', '', regex=True)
class_df.columns = ['업종대분류', '업종중분류', '업종소분류']

class_df.shape

(232, 3)

In [7]:
class_df.head()

,업종대분류,업종중분류,업종소분류
0,"농업, 임업 및 어업",농업,작물 재배업
10,"농업, 임업 및 어업",농업,축산업
17,"농업, 임업 및 어업",농업,작물재배 및 축산 복합농업
18,"농업, 임업 및 어업",농업,작물재배 및 축산 관련 서비스업
21,"농업, 임업 및 어업",농업,수렵 및 관련 서비스업


In [8]:
class_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232 entries, 0 to 1195
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   업종대분류   232 non-null    object
 1   업종중분류   232 non-null    object
 2   업종소분류   232 non-null    object
dtypes: object(3)
memory usage: 7.2+ KB


In [9]:
class_df.describe()

,업종대분류,업종중분류,업종소분류
count,232,232,232
unique,21,77,232
top,제조업,소매업; 자동차 제외,작물 재배업
freq,85,9,1


MERGE DFs

In [10]:
regex_corp_df = corp_df.copy()
regex_corp_df['regex_업종'] = regex_corp_df['업종'].replace(r'\W', '', regex=True)

regex_class_df = class_df.copy()
regex_class_df['regex_업종소분류'] = regex_class_df['업종소분류'].replace(r'[\W|ㆍ]', '', regex=True)

merged_df = pd.merge(regex_corp_df, regex_class_df, how='left', left_on='regex_업종', right_on='regex_업종소분류')
merged_df = merged_df.drop(['업종', 'regex_업종', 'regex_업종소분류'], axis=1)
merged_df.shape

(2542, 11)

In [11]:
merged_df.head()

,회사명,종목코드,주요제품,상장일,결산월,대표자명,홈페이지,지역,업종대분류,업종중분류,업종소분류
0,탈로스,434190,군용 리튬이온 이차전지,2022-10-24,12월,채재호,http://www.talos.or.kr,경기도,제조업,전기장비 제조업,일차전지 및 축전지 제조업
1,플라즈맵,405000,플라즈마 멸균기 및 표면처리기기,2022-10-21,12월,임유봉,http://plasmapp.com/,대전광역시,제조업,"의료, 정밀, 광학 기기 및 시계 제조업",의료용 기기 제조업
2,엔에이치스팩24호,437780,기업인수합병,2022-10-20,12월,이인직,NaN,서울특별시,금융 및 보험업,금융 및 보험관련 서비스업,금융 지원 서비스업
3,핀텔,291810,AI 영상 분석 솔루션 및 소프트웨어,2022-10-20,12월,김동기,NaN,경기도,정보통신업,출판업,소프트웨어 개발 및 공급업
4,하나금융25호스팩,435620,기업인수합병,2022-10-20,12월,남강욱,NaN,서울특별시,금융 및 보험업,금융 및 보험관련 서비스업,금융 지원 서비스업


In [12]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2542 entries, 0 to 2541
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   회사명     2542 non-null   object
 1   종목코드    2542 non-null   object
 2   주요제품    2527 non-null   object
 3   상장일     2542 non-null   object
 4   결산월     2542 non-null   object
 5   대표자명    2542 non-null   object
 6   홈페이지    2377 non-null   object
 7   지역      2541 non-null   object
 8   업종대분류   2542 non-null   object
 9   업종중분류   2542 non-null   object
 10  업종소분류   2542 non-null   object
dtypes: object(11)
memory usage: 238.3+ KB


In [13]:
merged_df.describe()

,회사명,종목코드,주요제품,상장일,결산월,대표자명,홈페이지,지역,업종대분류,업종중분류,업종소분류
count,2542,2542,2527,2542,2542,2542,2377,2541,2542,2542,2542
unique,2542,2542,2358,1886,9,2418,2355,22,17,64,161
top,탈로스,434190,기업인수합병,1976-06-30,12월,-,http://www.ibks.com,서울특별시,제조업,"전자 부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",소프트웨어 개발 및 공급업
freq,1,1,32,8,2482,5,3,1036,1540,286,158


SAVE

In [14]:
merged_df.to_csv('../../data/preprocessed/corp.csv', encoding='cp949', index=False)

In [15]:
mani_merged_df = merged_df.loc[merged_df['업종대분류'] == '제조업', :]
mani_merged_df.to_csv('../../data/preprocessed/mani_corp.csv', encoding='cp949', index=False)